In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Randomly generate a symmetric matrix
def generate_symmetric_matrix(dim):
    A = torch.rand(dim, dim, dtype=torch.double)
    return (A + A.t()) / 2

# Ensure the matrix is positive definite
def make_positive_definite(A):
    eigenvalues, _ = torch.linalg.eigh(A)
    if torch.all(eigenvalues > 0):
        return A
    else:
        return A + torch.eye(A.size(0)) * (torch.abs(eigenvalues.min()) + 0.1)

# Simplified MLP definition
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 5)
        self.fc2 = nn.Linear(5, output_dim)

    def forward(self, x):
        x = x.view(-1, 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x.view(2, 2)

# Generate symmetric positive 2x2 tensor
tensor = generate_symmetric_matrix(2)
tensor = make_positive_definite(tensor)

# Initialize model, criterion, and optimizer
model = SimpleMLP(4, 4)  # 4 input dimensions (2x2 matrix flattened) and 4 output dimensions
model.double()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 200

# Generate random input
random_input = torch.rand(2, 2, dtype=torch.double)
pre_train_output = model(random_input)

for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    outputs = model(random_input)
    
    loss = criterion(outputs, tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


print("\n\nNetwork to learn a random matrix -> 1x hardcoded matrix via MLP")
print('-'*10)
print(f'Input to network:\n{random_input}')
print(f'Pre-training model output:\n {pre_train_output}')
print('-'*10)
print(f"\nTrained Model's Output for the random input:\n{model(random_input)}")
print(f"Desired Tensor:\n{tensor}")


Epoch [20/200], Loss: 0.0311
Epoch [40/200], Loss: 0.0034
Epoch [60/200], Loss: 0.0003
Epoch [80/200], Loss: 0.0001
Epoch [100/200], Loss: 0.0000
Epoch [120/200], Loss: 0.0000
Epoch [140/200], Loss: 0.0000
Epoch [160/200], Loss: 0.0000
Epoch [180/200], Loss: 0.0000
Epoch [200/200], Loss: 0.0000


Network to learn a random matrix -> 1x hardcoded matrix via MLP
----------
Input to network:
tensor([[0.6944, 0.7120],
        [0.4758, 0.2865]], dtype=torch.float64)
Pre-training model output:
 tensor([[ 0.0769, -0.0997],
        [-0.3473,  0.2985]], dtype=torch.float64, grad_fn=<ViewBackward0>)
----------

Trained Model's Output for the random input:
tensor([[0.7344, 0.5279],
        [0.5280, 0.5487]], dtype=torch.float64, grad_fn=<ViewBackward0>)
Desired Tensor:
tensor([[0.7344, 0.5279],
        [0.5279, 0.5487]], dtype=torch.float64)


In [83]:
# Simplified MLP definition
class EigenvectorMLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(EigenvectorMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 5)
        self.fc2 = nn.Linear(5, output_dim)

    def forward(self, x):
        x = x.view(-1, 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x.view(2, 2)

# Generate random input and its eigenvectors
random_input = torch.rand(2, 2, dtype=torch.double)
_, eigenvectors = torch.linalg.eigh(random_input)

# Initialize model, criterion, and optimizer
model = EigenvectorMLP(4, 4)  # 4 input dimensions (2x2 matrix flattened) and 4 output dimensions
model.double()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 200

for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    outputs = model(random_input)
    
    loss = criterion(outputs, eigenvectors)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("\n\nNetwork to learn a random input -> 1x hardcoded eigenvector via MLP\nexcluding the eigh from network")
print('-'*10)
print('-'*10)
print(f"\nTrained Model's Output for the random input:\n{model(random_input)}")
print(f"True Eigenvectors:\n{eigenvectors}")


Epoch [20/200], Loss: 0.0038
Epoch [40/200], Loss: 0.0009
Epoch [60/200], Loss: 0.0002
Epoch [80/200], Loss: 0.0000
Epoch [100/200], Loss: 0.0000
Epoch [120/200], Loss: 0.0000
Epoch [140/200], Loss: 0.0000
Epoch [160/200], Loss: 0.0000
Epoch [180/200], Loss: 0.0000
Epoch [200/200], Loss: 0.0000


Network to learn a random input -> 1x hardcoded eigenvector via MLP
excluding the eigh from network
----------
----------

Trained Model's Output for the random input:
tensor([[ 0.4974, -0.8675],
        [-0.8675, -0.4974]], dtype=torch.float64, grad_fn=<ViewBackward0>)
True Eigenvectors:
tensor([[ 0.4974, -0.8675],
        [-0.8675, -0.4974]], dtype=torch.float64)


In [88]:
class EigenvectorMLPWithEigh(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(EigenvectorMLPWithEigh, self).__init__()
        self.fc1 = nn.Linear(input_dim, 5)
        self.fc2 = nn.Linear(5, output_dim)

    def forward(self, x):
        x = x.view(-1, 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x).view(2, 2)
        symmetric_matrix = (x + x.t()) / 2
        _, eigenvectors = torch.linalg.eigh(symmetric_matrix)
        return eigenvectors

# Generate random input and its eigenvectors
random_input = torch.rand(2, 2, dtype=torch.double)
_, target_eigenvectors = torch.linalg.eigh(random_input)

# Initialize model, criterion, and optimizer
model = EigenvectorMLPWithEigh(4, 4)  # 4 input dimensions (2x2 matrix flattened) and 4 output dimensions
model.double()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 5000

for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    outputs = model(random_input)
    
    loss = criterion(outputs, target_eigenvectors)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 150 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print('-'*10)
print(f"\nTrained Model's Output for the random input:\n{model(random_input)}")
print(f"True Eigenvectors:\n{target_eigenvectors}")


Epoch [150/5000], Loss: 1.3088
Epoch [300/5000], Loss: 0.8961
Epoch [450/5000], Loss: 0.6753
Epoch [600/5000], Loss: 0.4353
Epoch [750/5000], Loss: 0.2481
Epoch [900/5000], Loss: 0.1388
Epoch [1050/5000], Loss: 0.0710
Epoch [1200/5000], Loss: 0.0308
Epoch [1350/5000], Loss: 0.0111
Epoch [1500/5000], Loss: 0.0034
Epoch [1650/5000], Loss: 0.0009
Epoch [1800/5000], Loss: 0.0002
Epoch [1950/5000], Loss: 0.0000
Epoch [2100/5000], Loss: 0.0000
Epoch [2250/5000], Loss: 0.0000
Epoch [2400/5000], Loss: 0.0000
Epoch [2550/5000], Loss: 0.0000
Epoch [2700/5000], Loss: 0.0000
Epoch [2850/5000], Loss: 0.0000
Epoch [3000/5000], Loss: 0.0000
Epoch [3150/5000], Loss: 0.0000
Epoch [3300/5000], Loss: 0.0000
Epoch [3450/5000], Loss: 0.0000
Epoch [3600/5000], Loss: 0.0000
Epoch [3750/5000], Loss: 0.0000
Epoch [3900/5000], Loss: 0.0000
Epoch [4050/5000], Loss: 0.0000
Epoch [4200/5000], Loss: 0.0000
Epoch [4350/5000], Loss: 0.0000
Epoch [4500/5000], Loss: 0.0000
Epoch [4650/5000], Loss: 0.0000
Epoch [4800/50

In [71]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F

# # Randomly generate a symmetric matrix
# def generate_symmetric_matrix(dim):
#     A = torch.rand(dim, dim, dtype=torch.double)
#     return (A + A.t()) / 2

# # Ensure the matrix is positive definite
# def make_positive_definite(A):
#     eigenvalues, _ = torch.linalg.eigh(A)
#     if torch.all(eigenvalues > 0):
#         return A
#     else:
#         return A + torch.eye(A.size(0)) * (torch.abs(eigenvalues.min()) + 0.1)

# # MLP definition (same as before)
# class MLP(nn.Module):
#     def __init__(self):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(4, 9)
#         self.fc2 = nn.Linear(9, 2)

#     def forward(self, x):
#         x = x.view(-1, 4)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         a = (x + x.t()) * 0.5
#         v,x = torch.linalg.eigh(a)
#         return x[:,1], a
    
# # Generate symmetric positive 2x2 tensor
# tensor = generate_symmetric_matrix(2)
# tensor = make_positive_definite(tensor)

# # Calculate its second eigenvector (second eigenvalue is in the second column since they're sorted in ascending order by default)
# eigenvalues, eigenvectors = torch.linalg.eigh(tensor)
# second_eigenvector = eigenvectors[:, 1]

# # Initialize model, criterion, and optimizer
# model = MLP()
# model.double()
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# # Training loop
# num_epochs = 5000
# # Generate random input
# random_input = torch.rand(2, 2, dtype=torch.double)

# pre_train_ouput, pre_train_a = model(random_input)

# model.train()
# for epoch in range(num_epochs):
#     optimizer.zero_grad()
    
#     outputs, a = model(random_input)
    
#     loss = criterion(outputs, second_eigenvector)
#     loss.backward()
#     optimizer.step()

#     if (epoch+1) % 500 == 0:
#         print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# model.eval()
# print(f'pre-training output:\n{pre_train_ouput}')
# print(f'pre-training mlp output:\n{pre_train_a}')

# print('-'*10)

# print(f"\nTrained Model's Output for a random input:\n{model(random_input)[0]}")
# print(f"Second Eigenvector (Ground Truth):\n{second_eigenvector}")

# print('-'*10)

# print(f'What the mlp should learn:\n{tensor}')
# print(f'What the mlp learnt:\n{a}')


Epoch [500/100000], Loss: 1.9040
Epoch [1000/100000], Loss: 1.9040
Epoch [1500/100000], Loss: 1.9040
Epoch [2000/100000], Loss: 1.9040
Epoch [2500/100000], Loss: 1.9040
Epoch [3000/100000], Loss: 1.9040
Epoch [3500/100000], Loss: 1.9040
Epoch [4000/100000], Loss: 1.9040
Epoch [4500/100000], Loss: 1.9040
Epoch [5000/100000], Loss: 1.9040
Epoch [5500/100000], Loss: 1.9040
Epoch [6000/100000], Loss: 1.9040
Epoch [6500/100000], Loss: 1.9040
Epoch [7000/100000], Loss: 1.9040
Epoch [7500/100000], Loss: 1.9040
Epoch [8000/100000], Loss: 1.9040
Epoch [8500/100000], Loss: 1.9040
Epoch [9000/100000], Loss: 1.9040
Epoch [9500/100000], Loss: 1.9040
Epoch [10000/100000], Loss: 1.9040
Epoch [10500/100000], Loss: 1.9040
Epoch [11000/100000], Loss: 1.9040
Epoch [11500/100000], Loss: 1.9040
Epoch [12000/100000], Loss: 1.9040
Epoch [12500/100000], Loss: 1.9040
Epoch [13000/100000], Loss: 1.9040
Epoch [13500/100000], Loss: 1.9040
Epoch [14000/100000], Loss: 1.9040
Epoch [14500/100000], Loss: 1.9040
Epoch